###Ingestion del archivo "person.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
persons_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
persons_df = spark.read \
                .schema(persons_schema) \
                .json(f"{bronze_folder_path}/{v_file_date}/person.json")

In [0]:
persons_df.printSchema()

In [0]:
display(persons_df)

####Paso 2 - Renombrar las columnas y añadir nuevas columnas
- "personId" renombar a "person_id"
- Agregar las columnas "ingestion_date" y "environment"
- Agregar la columna "name" a partir de la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
persons_with_columns_df = add_ingestion_dat(persons_df) \
                            .withColumnRenamed("personId","person_id") \
                            .withColumn("environment", lit(v_environment)) \
                            .withColumn("name", 
                                        concat(
                                            col("personName.forename"), 
                                            lit(" "), 
                                            col("personName.surname")
                                            )
                                        ) \
                            .withColumn("file_date", lit(v_file_date))
display(persons_with_columns_df)

####Paso 3 - Escribir la salida en un formato "Parquet"

In [0]:
persons_final_df = persons_with_columns_df.drop(col("personName"))
display(persons_final_df)

####Paso 4 - Escribir la salida en un formato "Parquet"

In [0]:
merge_condition='tgt.person_id = src.person_id and tgt.file_date = src.file_date'
merge_delta_lake(persons_final_df, "movie_silver", "persons", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
select * from movie_silver.persons;

In [0]:
#persons_final_df.write.mode("overwrite").parquet(f"silver_folder_path/persons")
#persons_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.persons")

In [0]:
display(spark.read.format("delta").load("/mnt/mymoviehistory/silver/persons"))

In [0]:
dbutils.notebook.exit("Success")